This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='API_KEY.env')
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

RjAws_znvUZWFMZpbwyp


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {
    'api_key':API_KEY,
    'start_date':'2020-01-01',
    'end_date':'2020-01-02'
}
r = requests.get(url,params)
data_dict = r.json()
data_dict['dataset_data']['data'][0]

['2020-01-02',
 114.0,
 114.9,
 112.7,
 113.8,
 None,
 143464.0,
 16291666.0,
 None,
 None,
 None]

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(data_dict)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-01-01', 'end_date': '2020-01-02', 'frequency': 'daily', 'data': [['2020-01-02', 114.0, 114.9, 112.7, 113.8, None, 143464.0, 16291666.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params_2017 = {
    'api_key':API_KEY,
    'start_date':'2017-01-01',
    'end_date':'2017-12-31'
}

r = requests.get(url, params_2017)

In [6]:
data = r.json()
print(data['dataset_data']['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [7]:
# Question 3: Highest and lowest opening prices
idx = data['dataset_data']['column_names'].index('Open')
open_price = [day[idx] for day in data['dataset_data']['data'] if day[idx]]
print(max(open_price))
print(min(open_price))

53.11
34.0


In [8]:
# Question 4: Largest change in one day
idx_high = data['dataset_data']['column_names'].index('High')
idx_low = data['dataset_data']['column_names'].index('Low')

change = [day[idx_high]-day[idx_low] for day in data['dataset_data']['data']]
print(max(change))

Largest change in one day: 2.81


In [9]:
#Question 5: Largest change between 2 days
idx = data['dataset_data']['column_names'].index('Close')

#Extract closing price
closing = [day[idx] for day in data['dataset_data']['data']]

#Calculate change between everyday and the day before
change_two_days = [abs(closing[index]-closing[index-1]) for index, value in enumerate(closing) if index > 0]
print('The largest chagne between two days: {:.2f}'.format(max(change_two_days)))

The largest chagne between two days: 2.56


In [12]:
#Question 6: Average daily trading volume for the year
idx = data['dataset_data']['column_names'].index('Traded Volume')

#Extract trading volume
volume = [day[idx] for day in data['dataset_data']['data']]

#Since no third party packages are allowed, calculte the average the old fashioned way.
average_volume = sum(volume)/len(volume)
print(average_volume)

89124.33725490196


In [13]:
#Question 7: Median trading volume during the year
volume = sorted(volume)

#Find the median based on definition of median
if len(volume) % 2 == 1:
    median_volume = volume[len(volume)//2]
else:
    median_volume = (volume[len(volume)//2] + volume[len(volume)//2-1]) / 2
print(median_volume)

76286.0
